In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import t
from scipy.stats import norm
from scipy.stats import f
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Lectura del excel al pandas data frame
df = pd.read_csv('conjunto_de_datos_tmodulo_enut_2019.csv',encoding='latin-1')
#Agregar todas las columnas que se van a utilizar durante el analisis
colums = ['EDAD_V','SEXO','NIV','P5_1','P5_3_1','P5_3_2','P5_3_3','P5_3_4','P5_6_6','P5_7','P5_7A','P6_10_7','P6_13_3','P6_13_5','P6_13_6','P6_8_2','P6_6_3','P6_5_2','P6_4_3','P6_3_1']
#Tanto el vector siguiente como el for es para crear un vector que contenga todas las columnas 
#con horas utilizadas para el hogar
vec = ['P6_10A_7','P6_13A_3','P6_13A_5','P6_13A_6','P6_8A_2','P6_6A_3','P6_5A_2','P6_4A_3']
for x in range(len(vec)):
    tot_horas = 0
    tot_min = 0
    for i in range(4):
        nom_col = vec[x]
        nom_col = nom_col + '_' + str(i+1)
        colums.append(nom_col)
#Aqui se extrae solo las columnas que van a ser utilizadas
df_nuevo = df[colums]

In [3]:
#Esta es la limpieza de datos un para poder hacer la columna de horas trabajadas a la semana e ingreso por hora
df_nuevo['P6_13_3'] = df_nuevo['P6_13_3'].replace({np.nan: 0})
df_nuevo['P6_13_5'] = df_nuevo['P6_13_5'].replace({np.nan: 0})
df_nuevo['P6_13_6'] = df_nuevo['P6_13_6'].replace({np.nan: 0})
df_nuevo['P6_10_7'] = df_nuevo['P6_10_7'].replace({np.nan: 0})
df_nuevo['P6_8_2'] = df_nuevo['P6_8_2'].replace({np.nan: 0})
df_nuevo['P6_6_3'] = df_nuevo['P6_6_3'].replace({np.nan: 0})
df_nuevo['P6_5_2'] = df_nuevo['P6_5_2'].replace({np.nan: 0})
df_nuevo['P6_4_3'] = df_nuevo['P6_4_3'].replace({np.nan: 0})
df_nuevo['P6_13_3'] = df_nuevo['P6_13_3'].replace({2: 0})
df_nuevo['P6_13_5'] = df_nuevo['P6_13_5'].replace({2: 0})
df_nuevo['P6_13_6'] = df_nuevo['P6_13_6'].replace({2: 0})
df_nuevo['P6_10_7'] = df_nuevo['P6_10_7'].replace({2: 0})
df_nuevo['P6_8_2'] = df_nuevo['P6_8_2'].replace({2: 0})
df_nuevo['P6_6_3'] = df_nuevo['P6_6_3'].replace({2: 0})
df_nuevo['P6_5_2'] = df_nuevo['P6_5_2'].replace({2: 0})
df_nuevo['P6_4_3'] = df_nuevo['P6_4_3'].replace({2: 0})

In [4]:
#Limpieza de datos para poder generar la columna de horas trabajadas en la semana e ingreso de las personas
df_nuevo['P5_3_1'] = df_nuevo['P5_3_1'].replace({np.nan: '0'})
df_nuevo['P5_3_2'] = df_nuevo['P5_3_2'].replace({np.nan: '0'})
df_nuevo['P5_3_3'] = df_nuevo['P5_3_3'].replace({np.nan: '0'})
df_nuevo['P5_3_4'] = df_nuevo['P5_3_4'].replace({np.nan: '0'})
df_nuevo['P5_7'] = df_nuevo['P5_7'].replace({np.nan: '0'})
df_nuevo['P5_7'] = df_nuevo['P5_7'].replace({'99999': '0'})
df_nuevo['P5_7A'] = df_nuevo['P5_7A'].replace({np.nan: '-1'})
df_nuevo['P5_3_1'] = df_nuevo['P5_3_1'].astype(int)
df_nuevo['P5_3_2'] = df_nuevo['P5_3_2'].astype(int)
df_nuevo['P5_3_3'] = df_nuevo['P5_3_3'].astype(int)
df_nuevo['P5_3_4'] = df_nuevo['P5_3_4'].astype(int)
df_nuevo['P5_7'] = df_nuevo['P5_7'].astype(float)
df_nuevo['P5_7A'] = df_nuevo['P5_7A'].astype(int)
#Aqui lo que se hace es sumar cuantas horas a la semana trabajaron de luneas a viernes más las horas 
#que trabajaron los fines de semana
df_nuevo['tot_horas'] = df_nuevo['P5_3_1'] + df_nuevo['P5_3_3']
#Aqui lo que se hace es sumar cuantos minutos a la semana trabajaron de luneas a viernes más los minutos 
#que trabajaron los fines de semana
df_nuevo['tot_minutos'] = df_nuevo['P5_3_2'] + df_nuevo['P5_3_4']
#Aqui las horas que se habian sumado anteriormente se les suma los minutos totales divididos entre 60
df_nuevo['tot_horas'] = df_nuevo['tot_horas'] + (df_nuevo['tot_minutos']/60)

In [5]:
#Se aplican  los filtros para nuestra muestra
df_nuevo = df_nuevo.loc[df_nuevo['P5_7'] > 0] #Ingreso mayor a 0
df_nuevo = df_nuevo.loc[df_nuevo['P5_7A'] > 0] #Que las horas trabajadas sean mayores a 0
df_nuevo = df_nuevo.loc[df_nuevo['EDAD_V'] >= 15] #Edad mayor que o igual que 15 años
df_nuevo = df_nuevo.loc[df_nuevo['P5_1'] == 1] #Que tengan ISSSTE

In [6]:
#Como la columna de frecuencia de pagos no tiene valores que se puedan utilizar directamente para calcular el ingreso anual,
#se crea un vector paralelo que tenga valores útiles para calcular el salario anual.
def agregaColFrecPago(df):
    res = []
    for x in df:
        if x == 1:
            res.append(52)
        else:
            if x == 2:
                res.append(26)
            else:
                if x == 3:
                    res.append(12)
                else:
                    res.append(1)
    
    return res
#Este vector contiene multiples palabras que son el inicio de varias columnas que contienen la cantidad de horas 
#que cada persona le dedica a realizar labores de la casa
vec = ['P6_10A_7','P6_13A_3','P6_13A_5','P6_13A_6','P6_8A_2','P6_6A_3','P6_5A_2','P6_4A_3']
#Se van a inizializar 2 columnas en ceros para luego ir acumulando minutos y horas que cada persona le dedica al hogar
df_nuevo['tot_horas_casa'] = np.zeros(len(df_nuevo))
df_nuevo['tot_min_casa'] = np.zeros(len(df_nuevo))
#Este for consta de 2 partes, la primer sirve para hacer una limpieza rapida de las columnas y el segundo sirve para 
#acumular tanto las horas como minutos dedicados al hogar
for x in range(len(vec)):
#Aqui se hace la limpieza de datos
    for i in range(4):
        nom_col = vec[x]
        nom_col = nom_col + '_' + str(i+1)
        df_nuevo[nom_col] = df_nuevo[nom_col].replace({np.nan: 0})
        df_nuevo[nom_col] = df_nuevo[nom_col].replace({'': 0})
#Aqui se generan cadenas de carcteres que coiciden con columnas en la base de datos original y tienen horas dedicadas al hogar
    for i in range (2):
        nom_col = vec[x]
        nom_col = nom_col + '_' + str(i*2+1)
        df_nuevo['tot_horas_casa'] = df_nuevo['tot_horas_casa'] + df_nuevo[nom_col]
        nom_col = vec[x]
        nom_col = nom_col + '_' + str(i*2+2)
        df_nuevo['tot_min_casa'] = df_nuevo['tot_min_casa'] + df_nuevo[nom_col]


#Se generan las columnas tanto de horas trabajadas en la semana y el ingreso por hora de cada persona
x = agregaColFrecPago(df_nuevo['P5_7A'])
df_nuevo['semanal'] = ((x * df_nuevo['P5_7'])/48)
df_nuevo['ganancia_hora'] = df_nuevo['semanal']/df_nuevo['tot_horas']
df_nuevo = df_nuevo.loc[df_nuevo['P5_6_6'] == 1]

## Metodos para varianza y cosas extra

In [7]:
def s2df(df):
    
    n = len(df)
    media = np.mean(df)
    s2 = df - media
    s2 = np.dot(s2,s2)
    s2 = s2 / (n-1)
    
    return s2

def arrInterConfianza(metodo,arr,alfa):
    res = []
    for x in arr:
        
        res.append(metodo(x,alfa))
        
    return res

def arrEstimador(metodo,vec,col):
    res = []
    for x in vec:
        res.append(metodo(x[col]))
        
    return res

def pow_n(a):
    res = pow(a,1/2)
    
    return res

## Metodos para intervalos de confianza

In [8]:
#Metodo para obtener el intervalo de confianza de Miu con varianza desconocida
def intervaloDeConfVarDesc(df,alfa):
    
    res = []# arreglo vacio donde se van a guardar los resultados
    n = len(df) # se obtiene la longitud de la muestra
    grados_de_libertad = n - 1 # grados de libertad
    media = np.mean(df) # media del vector de datos
    s2 = s2df(df) # Se obtiene la S^2 con el vector de datos
    res_t = t.ppf(1-alfa/2, df=grados_de_libertad) # utilizando la prueba t se obtiene el valor del percentil 1-alfa/2
    sn = s2 / n # se calcula el resultado de dividir S^2/n
    sn = pow(sn,1/2) #se obtiene la raiz de S^2/n
    res.append((media - sn * res_t)) # limite inferior del intervalo
    res.append((media + sn * res_t)) # limite superior del intervalo
    
    return res

#Metodo para obtener el intervalo de confianza de la diferencia de Miu's con varianza desconocida
def intervaloDeConfDifMedias(x,alfa):
    #Basicamente los mismos pasos que el metedo anterior, solo que con dos vectores de datos
    res = []
    df_1 = x[0]
    df_2 = x[1]
    n1 = len(df_1)
    n2 = len(df_2)
    grados_de_libertad = n1 + n2 - 2
    media1 = np.mean(df_1)
    media2 = np.mean(df_2)
    dif = media1 - media2
    s21 = s2df(df_1)
    s22 = s2df(df_2)
    s1 = pow(s21,1/2)
    s2 = pow(s22,1/2)
    s2p = (((n1-1) * s21) + ((n2-1) * s22))/grados_de_libertad
    sp = pow(s2p,1/2)
    rn = (1/n1) + (1/n2)
    rn = pow(rn,1/2)
    res_t = t.ppf(1-alfa/2, df=grados_de_libertad)
    res.append((dif - res_t * sp * rn))
    res.append((dif + res_t * sp * rn))
    
    return res

#Metodo para cociente de varianzas
def intervaloDeConfCocienteDeVar(vec,alfa):
    
    res = []#Vector vacio para regresar el resultado
    # Se extraen los dos conjuntos de datos que se mandaron dentro de un vector
    df_1 = vec[0]
    df_2 = vec[1]
    
    #Se obtinene la longitud de cada vector datos 
    n1 = len(df_1)
    n2 = len(df_2)
    
    #Se obtienen las S^2 de cada vector de datos
    s21 = s2df(df_1)
    s22 = s2df(df_2)
    
    #Se obtiene el valor de F en el limite superior
    f_res_sup = f.ppf(1-(alfa/2),n1-1,n2-1)
    #Se obtiene el valor de F en el limite inferior
    f_res_inf = f.ppf(1-(alfa/2),n2-1,n1-1)
    f_res_inf = 1/f_res_inf
    
    #Cociente de las dos S^2 que se obtuvieron
    cociente = s22/s21
    
    #Se meten al vector de respuesta los limites del intervalos
    res.append(f_res_inf * cociente) #limite inferior
    res.append(f_res_sup * cociente) #limite superior
    
    return res


# Intervalos de confianza

## Intervalos para nivel educativo

In [9]:
#Se  generan los conjuntos de datos necesarios, en este caso es para nivel educativo
df_nula = df_nuevo.loc[df_nuevo['NIV'] == 0]
df_basica = df_nuevo.loc[df_nuevo['NIV'] > 0]
df_basica = df_basica.loc[df_basica['NIV'] < 4]
df_media = df_nuevo.loc[df_nuevo['NIV'] > 3]
df_media = df_media.loc[df_media['NIV'] < 8]
df_superior = df_nuevo.loc[df_nuevo['NIV'] > 7]

alfa = .05
arr_horas = [df_nula['tot_horas'],df_basica['tot_horas'],df_media['tot_horas'],df_superior['tot_horas']]
arr_pago_hora = [df_nula['ganancia_hora'],df_basica['ganancia_hora'],df_media['ganancia_hora'],df_superior['ganancia_hora']]
arr_inter_horas = arrInterConfianza(intervaloDeConfVarDesc,arr_horas,alfa)
arr_inter_pago_hora = arrInterConfianza(intervaloDeConfVarDesc,arr_pago_hora,alfa)

print('Intervalo de confinza para horas trabajadas con alfa del ' + str(alfa))
print('Intervalo de confinza para horas trabajadas con educacion nula : ' + str(arr_inter_horas[0]))
print('Intervalo de confinza para horas trabajadas con educacion básica : ' + str(arr_inter_horas[1]))
print('Intervalo de confinza para horas trabajadas con educacion media : ' + str(arr_inter_horas[2]))
print('Intervalo de confinza para horas trabajadas con educacion superior : ' + str(arr_inter_horas[3]))
print('-------------------------------------------------------------------------------------')
print('Intervalo de confinza para el ingreso por hora con alfa del ' + str(alfa))
print('Intervalo de confinza para el ingreso por hora con educacion nula : ' + str(arr_inter_pago_hora[0]))
print('Intervalo de confinza para el ingreso por hora con educacion básica : ' + str(arr_inter_pago_hora[1]))
print('Intervalo de confinza para el ingreso por hora con educacion media : ' + str(arr_inter_pago_hora[2]))
print('Intervalo de confinza para el ingreso por hora con educacion superior : ' + str(arr_inter_pago_hora[3]))

Intervalo de confinza para horas trabajadas con alfa del 0.05
Intervalo de confinza para horas trabajadas con educacion nula : [52.540485726846796, 60.53816595854646]
Intervalo de confinza para horas trabajadas con educacion básica : [51.246557520556806, 52.0477886534474]
Intervalo de confinza para horas trabajadas con educacion media : [49.90574090800223, 50.7715004713081]
Intervalo de confinza para horas trabajadas con educacion superior : [45.19034457172962, 45.98944389637001]
-------------------------------------------------------------------------------------
Intervalo de confinza para el ingreso por hora con alfa del 0.05
Intervalo de confinza para el ingreso por hora con educacion nula : [25.451599312538054, 34.07361260153309]
Intervalo de confinza para el ingreso por hora con educacion básica : [37.328681303547896, 40.45552288624239]
Intervalo de confinza para el ingreso por hora con educacion media : [44.66171375537651, 47.56049665281894]
Intervalo de confinza para el ingreso 

In [10]:
alfa = .1
arr_inter_horas = arrInterConfianza(intervaloDeConfVarDesc,arr_horas,alfa)
arr_inter_pago_hora = arrInterConfianza(intervaloDeConfVarDesc,arr_pago_hora,alfa)

print('Intervalo de confinza para horas trabajadas con alfa del ' + str(alfa))
print('Intervalo de confinza para horas trabajadas con educacion nula : ' + str(arr_inter_horas[0]))
print('Intervalo de confinza para horas trabajadas con educacion básica : ' + str(arr_inter_horas[1]))
print('Intervalo de confinza para horas trabajadas con educacion media : ' + str(arr_inter_horas[2]))
print('Intervalo de confinza para horas trabajadas con educacion superior : ' + str(arr_inter_horas[3]))
print('-------------------------------------------------------------------------------------')
print('Intervalo de confinza para el ingreso por hora con alfa del ' + str(alfa))
print('Intervalo de confinza para el ingreso por hora con educacion nula : ' + str(arr_inter_pago_hora[0]))
print('Intervalo de confinza para el ingreso por hora con educacion básica : ' + str(arr_inter_pago_hora[1]))
print('Intervalo de confinza para el ingreso por hora con educacion media : ' + str(arr_inter_pago_hora[2]))
print('Intervalo de confinza para el ingreso por hora con educacion superior : ' + str(arr_inter_pago_hora[3]))

Intervalo de confinza para horas trabajadas con alfa del 0.1
Intervalo de confinza para horas trabajadas con educacion nula : [53.194324141365904, 59.88432754402735]
Intervalo de confinza para horas trabajadas con educacion básica : [51.31098467689798, 51.98336149710622]
Intervalo de confinza para horas trabajadas con educacion media : [49.975359485353444, 50.70188189395689]
Intervalo de confinza para horas trabajadas con educacion superior : [45.25460108722411, 45.92518738087552]
-------------------------------------------------------------------------------------
Intervalo de confinza para el ingreso por hora con alfa del 0.1
Intervalo de confinza para el ingreso por hora con educacion nula : [26.15647914462921, 33.36873276944194]
Intervalo de confinza para el ingreso por hora con educacion básica : [37.5801112633094, 40.20409292648089]
Intervalo de confinza para el ingreso por hora con educacion media : [44.89481443358255, 47.3273959746129]
Intervalo de confinza para el ingreso por 

## Hombre vs mujeres

In [11]:
#Se  generan los conjuntos de datos necesarios, en este caso es genero
df_hombres = df_nuevo.loc[df_nuevo['SEXO'] == 1]
df_mujeres = df_nuevo.loc[df_nuevo['SEXO'] == 2]

alfa = .05 #alfa para la confiabilidad de los intervalos 

#Se generan dos vectores para meter las columnas con las que se quieren hacer los intervalos
arr_horas = [df_hombres['tot_horas'],df_mujeres['tot_horas']]
arr_pago_hora = [df_hombres['ganancia_hora'],df_mujeres['ganancia_hora']]
#Se manda a llamar a los metodos para generar intervalos y se guardan en variable para luego ser impresos los resultados
arr_inter_horas = arrInterConfianza(intervaloDeConfVarDesc,arr_horas,alfa)
arr_inter_pago_hora = arrInterConfianza(intervaloDeConfVarDesc,arr_pago_hora,alfa)

# Vectores para realiza diferencia de medias
x = [df_hombres['tot_horas'],df_mujeres['tot_horas']]
dif_media_horas = intervaloDeConfDifMedias(x,alfa)
x = [df_hombres['ganancia_hora'],df_mujeres['ganancia_hora']]
dif_media_ganancia = intervaloDeConfDifMedias(x,alfa)

#Se imprimen los resultados de los metodos anteriores
print('Intervalo de confinza para horas trabajadas con alfa del ' + str(alfa))
print('Intervalo de confinza para horas trabajadas de los hombres : ' + str(arr_inter_horas[0]))
print('Intervalo de confinza para horas trabajadas de las mujeres  : ' + str(arr_inter_horas[1]))
print('Intervalo de confinza para diferencia de horas promedio trabajadas : ' + str(dif_media_horas))
print('-------------------------------------------------------------------------------------')
print('Intervalo de confinza para el ingreso por hora con alfa del ' + str(alfa))
print('Intervalo de confinza para el ingreso por hora de los hombres : ' + str(arr_inter_pago_hora[0]))
print('Intervalo de confinza para el ingreso por hora de las mujeres : ' + str(arr_inter_pago_hora[1]))
print('Intervalo de confinza para diferencia de ingreso por hora promedio : ' + str(dif_media_ganancia))

Intervalo de confinza para horas trabajadas con alfa del 0.05
Intervalo de confinza para horas trabajadas de los hombres : [51.63751568887702, 52.286087326588905]
Intervalo de confinza para horas trabajadas de las mujeres  : [45.003917833802035, 45.66783887021761]
Intervalo de confinza para diferencia de horas promedio trabajadas : [6.148798163664933, 7.103048147781353]
-------------------------------------------------------------------------------------
Intervalo de confinza para el ingreso por hora con alfa del 0.05
Intervalo de confinza para el ingreso por hora de los hombres : [53.521016543503464, 59.40537245902544]
Intervalo de confinza para el ingreso por hora de las mujeres : [53.40977126165034, 56.256958507644406]
Intervalo de confinza para diferencia de ingreso por hora promedio : [-2.0890780628670917, 5.348737296101252]


In [12]:
alfa = .1

arr_inter_horas = arrInterConfianza(intervaloDeConfVarDesc,arr_horas,alfa)
arr_inter_pago_hora = arrInterConfianza(intervaloDeConfVarDesc,arr_pago_hora,alfa)

x = [df_hombres['tot_horas'],df_mujeres['tot_horas']]
dif_media_horas = intervaloDeConfDifMedias(x,alfa)

x = [df_hombres['ganancia_hora'],df_mujeres['ganancia_hora']]
dif_media_ganancia = intervaloDeConfDifMedias(x,alfa)

print('Intervalo de confinza para horas trabajadas con alfa del ' + str(alfa))
print('Intervalo de confinza para horas trabajadas de los hombres : ' + str(arr_inter_horas[0]))
print('Intervalo de confinza para horas trabajadas de las mujeres  : ' + str(arr_inter_horas[1]))
print('Intervalo de confinza para diferencia de horas promedio trabajadas : ' + str(dif_media_horas))
print('-------------------------------------------------------------------------------------')
print('Intervalo de confinza para el ingreso por hora con alfa del ' + str(alfa))
print('Intervalo de confinza para el ingreso por hora de los hombres : ' + str(arr_inter_pago_hora[0]))
print('Intervalo de confinza para el ingreso por hora de las mujeres : ' + str(arr_inter_pago_hora[1]))
print('Intervalo de confinza para diferencia de ingreso por hora promedio : ' + str(dif_media_ganancia))

Intervalo de confinza para horas trabajadas con alfa del 0.1
Intervalo de confinza para horas trabajadas de los hombres : [51.689661281066115, 52.233941734399814]
Intervalo de confinza para horas trabajadas de las mujeres  : [45.05730153902154, 45.614455164998105]
Intervalo de confinza para diferencia de horas promedio trabajadas : [6.225515052830172, 7.026331258616114]
-------------------------------------------------------------------------------------
Intervalo de confinza para el ingreso por hora con alfa del 0.1
Intervalo de confinza para el ingreso por hora de los hombres : [53.994122683248946, 58.93226631927996]
Intervalo de confinza para el ingreso por hora de las mujeres : [53.638704216487056, 56.02802555280769]
Intervalo de confinza para diferencia de ingreso por hora promedio : [-1.4911151956695337, 4.750774428903694]


### Cociente de varianzas

In [13]:
alfa = .05

x = [df_hombres['tot_horas'],df_mujeres['tot_horas']]
cociente_var = intervaloDeConfCocienteDeVar(x,alfa)
print('intervalo de confianza para cociente de varianzas del total de horas trabajadas en la semana' + str(cociente_var))

print('-----------------------------------------------------------------------------------------------------')

x = [df_hombres['ganancia_hora'],df_mujeres['ganancia_hora']]
cociente_var = intervaloDeConfCocienteDeVar(x,alfa)
print('intervalo de confianza para cociente de varianzas entre total ingreso por hora' + str(cociente_var))

intervalo de confianza para cociente de varianzas del total de horas trabajadas en la semana[0.697052508539755, 0.765301525044363]
-----------------------------------------------------------------------------------------------------
intervalo de confianza para cociente de varianzas entre total ingreso por hora[0.15573374735762488, 0.17098177381692398]


In [14]:
#En caso de querer replicar el experimento, se tiene que quietar el # de la siguiente linea de codigo para extraer
#la base de datos y correr el codigo de R con esa base de datos nueva (La base de datos es ENUT 2019 normal de INEGI)
#df_nuevo.to_csv('Nombre_del_archivo',index=False)